In [50]:
from bs4 import BeautifulSoup
import requests
import configparser
import pathlib
from collections import defaultdict
import pandas as pd
import re
import unicodedata
import nltk
from nltk import SnowballStemmer
from langdetect import detect
import html2text
nltk.download('stopwords')

In [43]:
class WebSitesConfigParser(): # naive class    
    def __init__(self, file_path):
        self._config_file = file_path
        self._sections = defaultdict(list)
        
    def __getitem__(self, index):
        return self._sections[index]
    
    def read(self):
        with open(self._config_file) as cf:
            lines = cf.readlines()
        for idx, line in enumerate(lines):
            if line.startswith("["):
                section_name = line.strip('\n[]')
                continue
            self._sections[section_name].append(line.strip('\n'))
        
    def get_sections(self):
        return list(self._sections.keys())

In [52]:
def get_web_content(url): #1
    try:
        url_res = requests.get(url).text
    except requests.TooManyRedirects as tmr:
        raise(tmr)
    else:
        html = url_res
        text = html2text.html2text(html)
        file = open("data/raw/site_{url}.txt".format(counter), 'w+')    
        file.write(text.strip('/n').encode('utf-8'))
        file.close()
    return text

def format_web_content(content): #2
    a,b = 'áéíóúü','aeiouu'
    trans = str.maketrans(a,b)
    stop_words = set(stopwords.words("spanish"))
    content = content.translate(trans)    
    tokens = re.findall(r"[a-zA-Z]{3,}", content)
    filtered_sentence = [w.lower() for w in tokens if not w in stop_words]    
    spanish = [word for word in filtered_sentence if detect(word) == 'es']
    return ' '.join(spanish)

def build_data_set(dict_):
    if not isinstance(dict_, dict):
        type_ = type(dict_)
        raise TypeError("Dictionary expected, {} found".format(type_))
    df = pd.DataFrame(columns = ['manually_assigned_tags', 'url', 'content'])
    for key in dict_.keys():
        for url in dict_[key]:
            try:
                content = format_web_content(get_web_content(url))
                entry = {'manually_assigned_tag': key, 'url': url, 'content': content}
                df.append(pd.DataFrame(entry))
            except requests.TooManyRedirects:
                print('Too many redirects exception raised. Ignoring website...')
                continue
    return df